# Royalty Data Audit — Load & Quality Checks

## Goal
This notebook loads simulated music royalty reporting data and performs initial data quality and completeness checks to support a royalty audit workflow.

The focus at this stage is to verify that reported usage data can be reliably reconciled with payment records before deeper financial analysis is performed.

In [ ]:
import pandas as pd

usage = pd.read_csv("../data/raw/usage_reports.csv")
rates = pd.read_csv("../data/raw/royalty_rates.csv")
payments = pd.read_csv("../data/raw/payments.csv")

print("USAGE")
display(usage.head())
usage.info()

print("\nRATES")
display(rates.head())
rates.info()

print("\nPAYMENTS")
display(payments.head())
payments.info()


In [ ]:
# Data quality checks

print("DATA QUALITY CHECKS")
print("Missing ISRCs:", usage["isrc"].isna().sum())
print("Negative or zero streams:", (usage["streams"] <= 0).sum())
print("Duplicate usage_report_id:", usage["usage_report_id"].duplicated().sum())


In [ ]:
# Audit-critical check: payment completeness

usage_paid = usage.merge(
    payments[["usage_report_id", "payment_status"]],
    on="usage_report_id",
    how="left"
)

unpaid_count = usage_paid["payment_status"].isna().sum()
unpaid_rate = unpaid_count / len(usage_paid)

print("USAGE ROWS WITH NO MATCHING PAYMENT:", unpaid_count)
print("UNPAID RATE (%):", round(unpaid_rate * 100, 2))

print("\nPAYMENT STATUS DISTRIBUTION (INCLUDING MISSING):")
print(
    usage_paid["payment_status"]
    .fillna("missing")
    .value_counts(normalize=True)
    .mul(100)
    .round(2)
)
